# Number Partitioning : prima programmazione

## Preparazione

Scaricate le librerie necessarie, mi sono  iscritta alla piattaforma **Leap** della D-Wave così da poter avere un token API e avere accesso alla QPU e agli Hybrid Solver. Per far sì che l'accesso non scadesse tra un mese, ho creato un cartella (pubblica) su github dove saranno pubblicati i vari codici che scriverò usando Ocean (era la condizione per avere uso gratuito ogni mese). Inoltre, la cartella git serviva anche a me per poter facilmente aggiornare i codici a prescindere dal pc su cui lavoro.

Ho voluto provare diversi approcci per vedere come funzionassero e quale fosse la sintassi corretta di essi; le riporto tutti i tentativi che ho fatto.

Ovviamente l'istanza che ho utilizzato è un *problema giocattolo*, ma ho pensato che per iniziare a prendere la mano con il linguaggio potesse essere la scelta migliore, data anche la facile generalizzazione di tutti i procedimenti che ho effettuato.

Altrettanto ovviamente, il programma che ho scritto non è ottimale però funziona. Ci sono solo alcuni dubbi su alcune funzioni di cui vorrei discutere con lei: i dubbi e le references sono riportate nel notebook quando tali funzioni sono chiamate.

In [1]:
# libraries
import numpy as np
import dimod as dd
from dwave.system import LeapHybridSampler
from dwave.system import DWaveSampler, EmbeddingComposite

## Dati iniziali

Dato un insieme, che nel nostro caso è $\{1,2,3,4,10\}$ (scelto affinché la differenza minima sia zero), possiamo scrivere la somma del primo sottoinsieme come
$$
    sum_1 = \sum_{j=1}^5 s_j x_j
$$
dove $s_j$ è il valore dell'elemento $j-$ esimo e $x_j\in\{0,1\}$ è la variabile binaria che indica se prendiamo $(x_j = 1)$ l'elemento $j$ oppure no $(x_j = 0)$.
<br>Analogamente, possiamo scrivere la somma del secondo sottoinsieme in relazione alla somma del primo:
$$
    sum_2 =   \sum_{j=1}^5 s_j  - \sum_{j=1}^5 s_j x_j 
$$
Dunque, la differenza tra i due sottoinsiemi è data da
$$
    d = tot -  2\sum_{j=1}^5 s_j x_j
$$
La formulazione QUBO è ottenuta supponendo di voler minimizzare $d^2$, ossia
$$
    d^2 = tot^2 + 4x^T Q x
$$
dove $Q$ è una matrice $5\times 5$ definita come
$$
    Q(i,j) = 
    \begin{cases}
        s_i(s_i - tot) & \text{se } i=j\\
        s_i s_j & \text{se } i\neq j
    \end{cases}
$$
definizione da cui possiamo osservare che $Q$ è simmetrica.

Dunque, dato che i problemi di minimizzazione possono essere analizzati a prescindere dalle costanti, il nostro problema QUBO si riduce a voler minimizzare la forma quadratica:
$$
    QUBO: \: \min x^T Q x
$$

In [2]:
# input set
num = [1,2,3,4,10]
tot = sum(num)

In [3]:
# QUBO matrix
Q = np.zeros((5,5))

for i in range(5):
    for j in range(i,5):
        if i == j: 
            Q[i][j] = num[i]*(num[i] - tot)
        else:
            Q[i][j] = num[i]*num[j]
            Q[j][i] = num[i]*num[j]

## Simulated Annealing

La prima prova che ho effettuato è stata tramite *simulated annealing*, usando il modulo `Neal` di `Ocean` per effettuare il sampling e `Pyqubo` per preparare le variabili e il modello da ottimizzare.

In [4]:
from pyqubo import Array
import neal

La classe $\underline{Array}$ è una classe equivalente all'omonima classe `Numpy`, su cui però sono definite le operazioni specifiche delle variabili di tipo SPIN o BINARY.

Creiamo dunque l'array di variabili binarie, di cui vogliamo trovare la combinazione ottimale.

In [5]:
x = Array.create('x', shape = (1,5), vartype = 'BINARY')

Per creare il modello QUBO o BQM da passare in input al SA, ci serve prima un modello intermedio ($ph\_model$) che sia in formato di operazioni tra numeri e variabili Binary. 

**nota**: molte variabili che uso solo temporaneamente o per passaggi intermedi sono chiamate $ph\_something$, a indicare che sono variabili segnaposto.

In [6]:
ph_model = 0

# operazioni
for i in range(5):
    for j in range(5):
        ph_model = ph_model + x[0,i]*Q[i][j]*x[0,j]

print(ph_model)

# creazione del modello con operazioni
model_pyqubo = ph_model.compile()

# conversione in BQM
bqm_pyqubo = model_pyqubo.to_bqm()
print(bqm_pyqubo)

(((Binary('x[0][4]') * -100.000000) * Binary('x[0][4]')) + ((Binary('x[0][4]') * 40.000000) * Binary('x[0][3]')) + ((Binary('x[0][4]') * 30.000000) * Binary('x[0][2]')) + ((Binary('x[0][4]') * 20.000000) * Binary('x[0][1]')) + ((Binary('x[0][4]') * 10.000000) * Binary('x[0][0]')) + ((Binary('x[0][3]') * 40.000000) * Binary('x[0][4]')) + ((Binary('x[0][3]') * -64.000000) * Binary('x[0][3]')) + ((Binary('x[0][3]') * 12.000000) * Binary('x[0][2]')) + ((Binary('x[0][3]') * 8.000000) * Binary('x[0][1]')) + ((Binary('x[0][3]') * 4.000000) * Binary('x[0][0]')) + ((Binary('x[0][2]') * 30.000000) * Binary('x[0][4]')) + ((Binary('x[0][2]') * 12.000000) * Binary('x[0][3]')) + ((Binary('x[0][2]') * -51.000000) * Binary('x[0][2]')) + ((Binary('x[0][2]') * 6.000000) * Binary('x[0][1]')) + ((Binary('x[0][2]') * 3.000000) * Binary('x[0][0]')) + ((Binary('x[0][1]') * 20.000000) * Binary('x[0][4]')) + ((Binary('x[0][1]') * 8.000000) * Binary('x[0][3]')) + ((Binary('x[0][1]') * 6.000000) * Binary('x[0][2

**Nota**: le stampe sono sicuramente da sistemare, così da renderle più chiare e facilmente interpretarle. Ho visto che esiste una libreria "PrettyPrint", ma devo sistemare qualche impostazione per poterla usare, quindi ho preferito dedicarmi ad altro in questo momento.

Possiamo dunque ora definire il nostro $sampler$ e cercare la combinazione di variabili ottimali per il nostro problema.

In [7]:
# simulated annealing
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm_pyqubo, num_reads=10)

# analisi output e stampa del migliore
samples = model_pyqubo.decode_sampleset(sampleset)
best_sample = min(samples, key=lambda s: s.energy)
print(best_sample.sample) 

{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1}


Osserviamo che la soluzione trovata è effetivamente ottimale: prendendo infatti $S_1 = \{1,2,3,4\}$ e $S_2 =\{10\}$ otteniamo una differenza nulla, esattamente ciò che ci aspettavamo.

Possiamo anche usare creare un modello QUBO partendo dal nostro modello intermedio usando la funzione dedicata:

In [8]:
# conversione in modello QUBO
qubo_pyqubo = model_pyqubo.to_qubo()
print(qubo_pyqubo)

({('x[0][4]', 'x[0][3]'): 80.0, ('x[0][3]', 'x[0][1]'): 16.0, ('x[0][0]', 'x[0][0]'): -19.0, ('x[0][3]', 'x[0][3]'): -64.0, ('x[0][1]', 'x[0][0]'): 4.0, ('x[0][4]', 'x[0][1]'): 40.0, ('x[0][4]', 'x[0][2]'): 60.0, ('x[0][3]', 'x[0][2]'): 24.0, ('x[0][2]', 'x[0][2]'): -51.0, ('x[0][3]', 'x[0][0]'): 8.0, ('x[0][2]', 'x[0][0]'): 6.0, ('x[0][2]', 'x[0][1]'): 12.0, ('x[0][4]', 'x[0][4]'): -100.0, ('x[0][1]', 'x[0][1]'): -36.0, ('x[0][4]', 'x[0][0]'): 20.0}, 0.0)


Osserviamo però che il sampler relativo al QUBO prende in input direttamente la matrice $Q$, che trasforma poi immediatamente in un tipo BQM (è possibile vederlo nel codice relativo pubblicato su [github](https://github.com/dwavesystems/dimod/blob/0.12.6/dimod/core/sampler.py#L268)).

In [9]:
# sample tramite QUBO
sampleset = sa.sample_qubo(Q)
sampleset

SampleSet(rec.array([([1, 1, 1, 1, 0], -100., 1)],
          dtype=[('sample', 'i1', (5,)), ('energy', '<f8'), ('num_occurrences', '<i4')]), Variables([0, 1, 2, 3, 4]), {'beta_range': [0.006931471805599453, 2.649158683274018], 'beta_schedule_type': 'geometric'}, 'BINARY')

Vorrei poi infatti discutere con lei di questo passaggio che viene effettuato: nella definizione di *sample_qubo* si passa a BQM per poi chiamare *sample*, ma in *sample* poi si ritorna a Ising o a QUBO (a seconda del tipo di variabili): non mi è chiaro quindi se effettivamente ci sia una qualche differenza oppure no.

Riporto le references:
<br> [sampler](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/sampler_composites/generated/dimod.Sampler.sample.html#dimod.Sampler.sample)
<br> [sample_qubo](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/sampler_composites/generated/dimod.Sampler.sample_qubo.html#dimod.Sampler.sample_qubo)

Ho poi scoperto, leggendo la documentazione, che la libreria `Neal` sarà eliminata nei prossimi aggiornamenti, dato che è stata sostituita da un `d-wave-sampler`: il procedimento resta in ogni caso lo stesso.

In [10]:
from dwave.samplers import SimulatedAnnealingSampler

# BQM
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm_pyqubo, num_reads=10)

samples = model_pyqubo.decode_sampleset(sampleset) # funzione pyqubo
best_sample = min(samples, key=lambda s: s.energy)
print("BQM:")
print(best_sample.sample) 

# QUBO
sampleset = sampler.sample_qubo(Q)
samples = model_pyqubo.decode_sampleset(sampleset)
best_sample = min(samples, key=lambda s: s.energy)
print("QUBO")
print(best_sample.sample) 

BQM:
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1}
QUBO
{'x[0][4]': 0, 'x[0][3]': 0, 'x[0][2]': 0, 'x[0][1]': 0, 'x[0][0]': 1}


## Quantum Annealing + Hybrid Solver

Sono poi passata ad usare esclusivamente la libreria `Ocean`, facendo così delle prove con i risolutori ibridi e con la loro QPU.

Iniziamo dunque creando il BQM su cui lavorare:

In [11]:
bqm_wave = dd.BQM.from_qubo(Q)
bqm_wave

BinaryQuadraticModel({0: -19.0, 1: -36.0, 2: -51.0, 3: -64.0, 4: -100.0}, {(1, 0): 4.0, (2, 0): 6.0, (2, 1): 12.0, (3, 0): 8.0, (3, 1): 16.0, (3, 2): 24.0, (4, 0): 20.0, (4, 1): 40.0, (4, 2): 60.0, (4, 3): 80.0}, 0.0, 'BINARY')

Procediamo ora con il sampling tramite QPU: per usare il processore quantistico è però necessario effettuare un *embedding*, ossia mappare le variabili del nostro problema sulla topologia specifica della QPU. 

La funzione qui usata è quella più generale e si basa sull'euristica, senza presuppore una conoscenza approfondita della struttura del problema

In [12]:
sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm_wave, num_reads=100) # submit to a QPU 
print("A titolo di esempio, riportiamo tutto il sample_set")
print(sampleset)

print("I tempi sono da intendere da microsecondi:")
sampleset.info

A titolo di esempio, riportiamo tutto il sample_set
    0  1  2  3  4 energy num_oc. chain_.
0   0  0  0  0  1 -100.0      15     0.0
1   1  1  1  1  0 -100.0      24     0.0
2   1  0  0  0  1  -99.0      19     0.0
3   0  1  1  1  0  -99.0      16     0.0
4   1  0  1  1  0  -96.0      12     0.0
5   0  1  0  0  1  -96.0       2     0.0
6   1  1  0  0  1  -91.0       4     0.0
7   0  0  1  1  0  -91.0       3     0.0
8   0  0  1  0  1  -91.0       1     0.0
9   1  1  0  1  0  -91.0       3     0.0
10  1  0  0  1  1  -75.0       1     0.0
['BINARY', 11 rows, 100 samples, 5 variables]
I tempi sono da intendere da microsecondi:


{'timing': {'qpu_sampling_time': 9492.0,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 54.38,
  'qpu_access_time': 25254.77,
  'qpu_access_overhead_time': 1083.23,
  'qpu_programming_time': 15762.77,
  'qpu_delay_time_per_sample': 20.54,
  'total_post_processing_time': 517.0,
  'post_processing_overhead_time': 517.0},
 'problem_id': '43c4c034-cc98-4bdb-93f4-a56046103821'}

Nuovamente, possiamo implementare la stessa soluzione direttamente tramite QUBO:

In [13]:
sampleset = EmbeddingComposite(DWaveSampler()).sample_qubo(Q, num_reads=100)
print(sampleset)

print("I tempi sono da intendere da microsecondi:")
sampleset.info

    0  1  2  3  4 energy num_oc. chain_.
0   0  0  0  0  1 -100.0      13     0.0
1   1  1  1  1  0 -100.0      12     0.0
2   0  1  1  1  0  -99.0      16     0.0
3   1  0  0  0  1  -99.0      24     0.0
4   0  1  0  0  1  -96.0      11     0.0
5   1  0  1  1  0  -96.0       3     0.0
6   0  0  1  0  1  -91.0       4     0.0
7   1  1  0  1  0  -91.0       4     0.0
8   0  0  1  1  0  -91.0       6     0.0
9   1  1  0  0  1  -91.0       2     0.0
10  0  0  0  1  1  -84.0       2     0.0
11  1  1  1  0  0  -84.0       3     0.0
['BINARY', 12 rows, 100 samples, 5 variables]
I tempi sono da intendere da microsecondi:


{'timing': {'qpu_sampling_time': 10732.0,
  'qpu_anneal_time_per_sample': 20.0,
  'qpu_readout_time_per_sample': 66.78,
  'qpu_access_time': 26493.97,
  'qpu_access_overhead_time': 1197.03,
  'qpu_programming_time': 15761.97,
  'qpu_delay_time_per_sample': 20.54,
  'total_post_processing_time': 1933.0,
  'post_processing_overhead_time': 1933.0},
 'problem_id': 'c3593efd-d249-4c94-bd83-d0d47a55a623'}

Anche l'Hybrid Sampler segue gli stessi procedimenti:

In [14]:
sampler = LeapHybridSampler(solver={'category': 'hybrid'})    

# BQM
sampleset_hybrid = sampler.sample(bqm_wave)  
print("BQM") 
print(sampleset_hybrid)    

print("I tempi sono da intendere da microsecondi:")
sampleset_hybrid.info

BQM
   0  1  2  3  4 energy num_oc.
0  0  0  0  0  1 -100.0       1
['BINARY', 1 rows, 1 samples, 5 variables]
I tempi sono da intendere da microsecondi:


{'qpu_access_time': 128576,
 'charge_time': 2982837,
 'run_time': 2982837,
 'problem_id': '8474cc2c-0837-4a87-a9b2-d51b03bdddd2'}

In [15]:
# QUBO
sampleset_hybrid = sampler.sample_qubo(Q)
print("QUBO")
print(sampleset_hybrid)  

print("I tempi sono da intendere da microsecondi:")
sampleset_hybrid.info

QUBO
   0  1  2  3  4 energy num_oc.
0  0  0  0  0  1 -100.0       1
['BINARY', 1 rows, 1 samples, 5 variables]
I tempi sono da intendere da microsecondi:


{'qpu_access_time': 96323,
 'charge_time': 2998020,
 'run_time': 2998020,
 'problem_id': 'f8c8e6f7-cb99-424d-b507-35335a915b22'}